In [ ]:
# ! pip install --upgrade pip
# ! pip install --user numpy scipy matplotlib pyhf cabinetry uproot pandas

In [ ]:
import numpy as np
import cabinetry
import pyhf
import json
import uproot
import pandas as pd
from pathlib import Path

cabinetry.__version__

# $B^+ \to K^+ \pi^0$ example 

Here we want to study the fitting of the $B^+ \to K^+ \pi^0$ channel, that you reconstructed in the basf2 workshop. There are some decisions to make before constructing a statistical model:

* **Fitting variable**: The choice of fitting variable is crucial. Ideally, one choses a variable where distributions of signal and background look very different. This gives more power to distinguish the two during the fitting procedure. Here we chose $\Delta E$, which is the difference between the energy of the $B$ and half the center of mass energy.

* **Binning**: The choice of binning is an important one. If the binning is too fine, we won't have enough statistics. If the binning is too wide, we lose information that might be important to us. In this example we chose a binning of 20 equally distributed bins in the range $-0.4 \geq \Delta E \geq 0.4$.

* **Samples**: We have to do a good job at modelling our data with MC samples. We include samples of different nature, to build a template. The benefit of keeping the samples separate is that we can modify them individually, and assign different modifiers to them. Here we made the following choice of samples:
  * **signal**: This includes the signal MC
  * **qqbar**: This includes $e^+e^- \to q \bar q$ background events, where we only include $s \bar s$ and $c \bar c$, as these are the dominant ones.
  * **BBbar**: This includes $B^+ B^-$ and $B^0 \bar B ^0$ background events.
  * **misID**: This includes events where the kaon is misidentified as pion.

# Building the model

We will use `Cabinetry` ([documentation](https://cabinetry.readthedocs.io/en/latest/)) to build our `pyhf` model. We will see, this comes with many convenience functions to make our life easier. For example, `Cabinetry` has the nice feature that it can crate `pyhf` models from `root` files. 

First we define the binning for our fitting variable.

In [ ]:
bins = np.linspace(-0.4, 0.4, 20 + 1)

We define `cabinetry` models vie a `config` dictionary, containing different settings.

First, we give our measurement a name, define a parameter of interest (POI), and input path containing the `root` files and a histogram folder, where cabinetry automatically saves the histogram yields.

In [ ]:
cabinetry.set_logging()

BASE = Path('ntuples')

config = {
   'General':{
      'Measurement': 'B2Kpi0',
      'POI': 'mu',
      'InputPath': str(BASE / '{SamplePath}'), # wildcard for samples
      'HistogramFolder': 'histograms/'

   }
}

Before we continue, we want to calculate the correct normalizations for the individual samples. Normally, one has to scale according to luminosity. Since we use MC to simulate data, we simplify our lives by comparing to the generic MC files and summing the event weights. Here we only use the PID weights in the `data_MC_ratio` entry, calculated using the [systematics framework](https://syscorrfw.readthedocs.io/en/latest/index.html). How we assign them and how their uncertainties are calculated is described in the `pid-weights.ipynb` notebook. If you have time at the end, feel free to have a look at the notebook.

First, let us load all the signal and background ntuples. We use a mix of reconstructed MC samples as the data in this example (charged and mixed $B \bar B$, $s \bar s$ and $c \bar c$).

In [ ]:
with uproot.open({'ntuples/data_ssbar.root': 'B'}) as tree:
    dat_ssb = tree.arrays(tree.keys(), library="pd")
    
with uproot.open({'ntuples/data_ccbar.root': 'B'}) as tree:
    dat_ccb = tree.arrays(tree.keys(), library="pd")
    
with uproot.open({'ntuples/data_charged.root': 'B'}) as tree:
    dat_cha = tree.arrays(tree.keys(), library="pd")
    
with uproot.open({'ntuples/data_mixed.root': 'B'}) as tree:
    dat_mix = tree.arrays(tree.keys(), library="pd")
    
with uproot.open({'ntuples/signal.root': 'B'}) as tree:
    sig = tree.arrays(tree.keys(), library="pd")
    
with uproot.open({'ntuples/ssbar.root': 'B'}) as tree:
    ssb = tree.arrays(tree.keys(), library="pd")
    
with uproot.open({'ntuples/ccbar.root': 'B'}) as tree:
    ccb = tree.arrays(tree.keys(), library="pd")

with uproot.open({'ntuples/charged.root': 'B'}) as tree:
    cha = tree.arrays(tree.keys(), library="pd")
    
with uproot.open({'ntuples/mixed.root': 'B'}) as tree:
    mix = tree.arrays(tree.keys(), library="pd")
    
with uproot.open({'ntuples/misID.root': 'B'}) as tree:
    mis = tree.arrays(tree.keys(), library="pd")

Before summing the weights for the corresponding samples, we apply additional cuts. We combine the $s \bar s$ and $c\bar c$ backgrounds to a common $q \bar q$ background and the charged and mixed $B \bar B$ to a common $B \bar B$ background. 

In [ ]:
w = 'data_MC_ratio'

global_cuts = '(abs(B_deltaE)<0.4) & (B_R2<0.45) & (B_cosTBTO<0.8)'

signal_cuts = global_cuts + '& (B_isSignal==1)'
signal_norm = dat_cha.query(signal_cuts).sum()[w] / sig.query(signal_cuts).sum()[w]
print('signal norm:', signal_norm)

qqbar_cuts = global_cuts
qqbar_norm = (dat_ssb.query(global_cuts).sum()[w] + dat_ccb.query(global_cuts).sum()[w]) / (ssb.query(global_cuts).sum()[w] + ccb.query(global_cuts).sum()[w])
print('qqbar norm:', qqbar_norm)

BBbar_cuts = global_cuts + '& (B_mcErrors>0) & (B_mcErrors!=128)'
BBbar_norm = (dat_cha.query(BBbar_cuts).sum()[w] + dat_mix.query(BBbar_cuts).sum()[w]) / (cha.query(BBbar_cuts).sum()[w] + mix.query(BBbar_cuts).sum()[w])
print('BBbar norm:', BBbar_norm)

misID_cuts = global_cuts + '& (B_mcErrors==128)'
misID_norm = dat_cha.query(misID_cuts).sum()[w] / mis.query(misID_cuts).sum()[w]
print('misID norm:', misID_norm)


In the `Regions` setting, we tell `cabinetry` which variable in the `root` files it should load and define our signal region via the cut $| \Delta E | < 0.4 GeV$, applied through a `Filter` entry. It is a list, because we can use events from more than one phase space region. Additionally, we define the binning.

Next we can define our `Samples` in a list, where we specify the name of each sample, the `root` file in the `InputPath`, the `Tree` and whether it is data or not. We can also pass a list of files as `SamplePath` and `cabinetry` will combine the files for us. Further, we can add weights to each sample, which can be a numeric scalar, applied to the full sample as a global weight, a column name in the `root` file for event-based weights, or a product of such. Also, to the samples can be filtered additionally.

In [ ]:
config.update({
   'Regions':[
      {
         'Name': 'signal_region',
         'Filter': global_cuts,
         'Variable': 'B_deltaE',                               # which variable we bin histograms in
         'Binning': list(bins)
      }
   ]
})

config.update({
   'Samples':[
      {
         'Name': 'Data',
         'Tree': 'B',
         'SamplePath': ['data_ssbar.root', 'data_ccbar.root', 'data_charged.root', 'data_mixed.root'],
         'Weight': f'data_MC_ratio',
         'Data': True                                          # observed data is handled differently, need to distinguish
      },
      {
         'Name': 'signal',
         'Tree': 'B',
         'Filter': signal_cuts,
         'SamplePath': 'signal.root',
         'Weight': f'{signal_norm}*data_MC_ratio'             
      },
      {
         'Name': 'qqbar',
         'Tree': 'B',
         'Filter': qqbar_cuts,
         'SamplePath': ['ssbar.root', 'ccbar.root'],
         'Weight': f'{qqbar_norm}*data_MC_ratio'
      },
       {
         'Name': 'BBbar',
         'Tree': 'B',
         'Filter': BBbar_cuts,
         'SamplePath': ['charged.root', 'mixed.root'],
         'Weight': f'{BBbar_norm}*data_MC_ratio'
      },
       {
         'Name': 'misID',
         'Tree': 'B',
         'Filter': misID_cuts,
         'SamplePath': 'misID.root',
         'Weight': f'{misID_norm}*data_MC_ratio',
      }
   ]
})

Lastly, we can add some modifiers. First, we add some normalization factors for the signal. Here, we specify our `POI`, the signal normalization `mu`.

In [ ]:
config.update({
   'NormFactors':[
      {
         'Name': 'mu',
         'Samples': 'signal',    # we want this parameter to scale the signal
         'Nominal': 1,
         'Bounds': [-10, 10]
      },
   ]
})

`cabinetry` lets us validate our `config`,


In [ ]:
cabinetry.configuration.validate(config)

Additionally, we can print an overview. We see that we have 5 samples, 1 region, 1 normalisation factor and no systematics so far. 

In [ ]:
cabinetry.configuration.print_overview(config)

## Creating the histograms

Given that our validation succeeds, we can `build` the histrograms for our model. This will create the hisrograms from the `root` files and save them into the `HistogramFolder`.

In [ ]:
cabinetry.templates.build(config, method='uproot')

You can also provide existing histograms you built yourself for `cabinetry` to use, see the [cabinetry-tutorials](https://github.com/cabinetry/cabinetry-tutorials) repository for an example.

`Cabinetry` also allows us to apply post-processing to our histograms, which consists of a fix for `NaN` statistical uncertainties and optional smoothing. It will create new histogram files in the `HistogramFolder` folder with the `*_modified.npz` ending.

In [ ]:
cabinetry.templates.postprocess(config)

We can now visualise what we produced:

In [ ]:
_ = cabinetry.visualize.data_mc_from_histograms(config)

`cabinetry` will automatically save this image in a `/figures` folder.

# Adding systematics

We now want to make our model more realistic by adding systematic uncertainties.

## Background normalization

We already normalized our model correctly to the data above. Our parameter of interest, the signal strength, is an unconstrained normalization. We want to constrain the background normalizations, since we usually believe that our modelling of these is correct within a certain uncertainty. The tightness of these constraints is very case dependent. Here we chose a very liberal backround normalization constraint of 50%.

In [ ]:
norm_sys = [
      {
         'Name': 'norm_qqbar',
         'Samples': 'qqbar',
         "Up": {"Normalization": 0.5},
         "Down": {"Normalization": -0.5},
         "Type": "Normalization"
      },
      {
         'Name': 'norm_BBbar',
         'Samples': 'BBbar',
         "Up": {"Normalization": 0.5},
         "Down": {"Normalization": -0.5},
         "Type": "Normalization"
      },
      {
         'Name': 'norm_misID',
         'Samples': 'misID',
         "Up": {"Normalization": 0.5},
         "Down": {"Normalization": -0.5},
         "Type": "Normalization"
      },
   ]


## Tracking efficiency

We rerun the reconstruction, removing some tracks. The resulting ntuples contain slightly different yields which tell us about the systematic uncertainty due to the tracking efficiency. 

In `cabinetry`, adding these uncertainties is very easy. We just specify the up- and down-variations via the new `root` file. Here we tell the configuration to symmetrize the variation. Other settings, such as `Tree, Weight, Variable, Filter,...` are inherited (see https://cabinetry.readthedocs.io/en/latest/config.html#template).

The `NormPlusShape` corresponds to a `histosys` modifier plus a `normsys` modifier. The `histosys` modifier takes care of the normalized shape variation due to our new samples and the `normsys` modifier takes care of the normalization differences of our nomial sample and the the modified one. `Cabinetry` will give the modifiers the same name, which will tell `pyhf` that these modifiers are correlated. Hence we will only see 1 nuisance parameter per sample in this case.



In [ ]:
tracking_sys = [
      {
         'Name':'TrackingEfficiency_signal',
         'Up': {
               'SamplePath': 'signal_rmt.root'
               },
         'Down': {'Symmetrize': True},
         'Samples': 'signal',
         'Type': 'NormPlusShape'
      },
      {
         'Name':'TrackingEfficiency_qqbar',
         'Up': {
               'SamplePath': ['ssbar_rmt.root', 'ccbar_rmt.root']
               },
         'Down': {'Symmetrize': True},
         'Samples': 'qqbar',
         'Type': 'NormPlusShape'
      },
      {
         'Name':'TrackingEfficiency_BBbar',
         'Up': {
               'SamplePath': ['charged_rmt.root', 'mixed_rmt.root'] 
               },
         'Down': {'Symmetrize': True},
         'Samples': 'BBbar',
         'Type': 'NormPlusShape'
      },
      {
         'Name':'TrackingEfficiency_misID',
         'Up': {
               'SamplePath': 'misID_rmt.root'
               },
         'Down': {'Symmetrize': True},
         'Samples': 'misID',
         'Type': 'NormPlusShape'
      }
   ]

Let us add the normalization and tracking efficiency systematics to our configuration.

In [ ]:
config.update({
   'Systematics': norm_sys + tracking_sys
})
cabinetry.templates.build(config, method='uproot')

## PID systematics

The PID systematics are not event-dependent, but bin-dependent. Adding bin-by-bin weights is not a feature that is implemented in `cabinetry` yet, so it is easier to add these systematics to the `pyhf` model directly. We will discuss this shortly, after we used cabinetry to build our `pyhf` model.

# Building a `pyhf` workspace

We now construct a `pyhf` workspace, which contains everything to build our likelihood function. This can also be used as an input file for `pyhf`. 

In [ ]:
workspace_path = 'b2kpi_workspace.json'
spec = cabinetry.workspace.build(config)

## Adding PID systematics

The systematics on the PID weights are stored in the `pid_systematics.csv` file. This is nothing but a table, split up by sample and fitting bin.
Hence, the systematics depend on our choice of binning. The type `u` corresponds to uncorrelated uncertainties and the `c` types correspond to correlated ones.

In [ ]:
pid_sys = pd.read_csv('pid-tables/pid_systematics.csv')
pid_types = pid_sys.filter(regex=r'type')
pid_sig = pid_sys.filter(regex=r'signal*')
pid_sys

Now we add the systematics directly to the `pyhf` model. The uncorrelated part, we add as a `staterror` type, because this reduced the number of nuisance parameters. The correlated uncertainties we add as `histosys` modifiers, by specifying the up and down variation of the yields. 

In [ ]:
def add_quadrature(a,b):
    return np.sqrt(np.add(np.power(a, 2),np.power(b, 2)).tolist())
    
# loop over variations
for _ , var in pid_sys.iterrows():
    # loop over samples
    for sam in spec['channels'][0]['samples']:
        name   = sam['name']
        yields = sam['data']
        variation = var.filter(regex=rf'{name}*').to_numpy()
        #apply variations according to type
        if var.type=='u':
            sam['modifiers'].append(
                {   
                    'name': f'pid_sys_{var.type}',
                    'type': 'staterror',
                    'data': list(variation)
                }
            )
        else:
            sam['modifiers'].append(
                {   
                    'name': f'pid_sys_{var.type}',
                    'type': 'histosys',
                    'data': {
                            'hi_data': list(yields*(1+variation)),
                            'lo_data': list(yields*(1-variation))
                            }
                }
            )

We can now print our workspace and inspect it.

In [ ]:
cabinetry.workspace.save(spec, workspace_path)
print(json.dumps(spec, sort_keys=True, indent=4))

## Model structure

It can be helpful to visualize the modifier structure of the statistical model we have built to catch potential issues. The `visualize.modifier_grid` function creates a figure showcasing the information about which modifiers (indicated by color) act on which region and sample when a given parameter (on the horizontal axis) is varied.

In [ ]:
cabinetry.visualize.modifier_grid(pyhf.Workspace(spec).model())

# Performing statistical inference

To perform inference, we need two things: a probability density function (pdf), or `model`, and data to fit it to. Both are derived from the workspace specification.

In [ ]:
model, data = cabinetry.model_utils.model_and_data(spec)

We see that all the modifiers that we defined for our model appear here.

## Validating the tracking efficiency systematics 

First let us check that we indeed only get 1 nuisance parameter per sample for the `TrackingEfficiency` modifier:

In [ ]:
model.config.par_names

Next we want to verify the numeric variations for the signal region.
To check these variations, we load the signal region Up-variation histogram for the `TrackingEfficiency`, which corresponds to the yields of the modified sample. To compare, we also load the nominal yields.

In [ ]:
tracking_nominal = np.load('histograms/signal_region_signal.npz')['yields']
tracking_modified = np.load('histograms/signal_region_signal_TrackingEfficiency_signal_Up.npz')['yields']

First let us check the `normsys` variations. We expect:

In [ ]:
spec['channels'][0]['samples'][0]['modifiers'][2]

Here `hi` and `lo` are just labels for the numbers we interpolate between. It does not matter if `lo` is larger than `hi`.

We can calculate these values as the ratio of the total number of (possibly weighted) events. Make sure you understand the origin of these numbers.

In [ ]:
sum(tracking_modified)/sum(tracking_nominal)

In [ ]:
(2*sum(tracking_nominal) - sum(tracking_modified))/sum(tracking_nominal)

Now let us check the `histosys` entries. We expect

In [ ]:
spec['channels'][0]['samples'][0]['modifiers'][3]

We calculate these numbers by calculating the differenced of the nominal yields to the correctly scaled modified ones.

In [ ]:
scale = sum(tracking_modified)/sum(tracking_nominal)
tracking_modified/scale

In [ ]:
2*tracking_nominal - tracking_modified / scale

Can you explain the origin of these formulae?

## Maximum likelihood estimate

Let's fit our model to data to obtain the maximum likelihood estimate (MLE).

In [ ]:
fit_results = cabinetry.fit.fit(model, data)

The fit converged, and we see the best-fit parameter results reported. The results are stored in a named tuple. This allows for easy access of the results. 

In [ ]:
for label, result, unc in zip(fit_results.labels, fit_results.bestfit, fit_results.uncertainty):
    print(f'{label}: {result:.3f} +/- {unc:.3f}')

It is helpful to visualize the fit results. Let's start with the pull plot showing us the constrained best-fit parameter results.

In [ ]:
cabinetry.visualize.pulls(fit_results, exclude=['mu']+[l for l in fit_results.labels if 'pid_sys_u' in l])

The parameter correlation matrix has a handy `pruning_threshold` setting to filter out parameters that are not highly correlated with others.

In [ ]:
cabinetry.visualize.correlation_matrix(fit_results, pruning_threshold=0.2)

Let us look at the post-fit result. This is as easy as passing `fit_results` to `cabinetry.model_utils.prediction`.

In [ ]:
model_pred_postfit = cabinetry.model_utils.prediction(model, fit_results=fit_results)
_ = cabinetry.visualize.data_mc(model_pred_postfit, data, config=config)

Yield tables can also be created from a model prediction, and compared to data. Optional keyword arguments control whether yields per bin are shown (`per_bin=True`, default) and whether bins summed per region are shown (`per_channel=True`, disabled by default). The yield table is also saved to disk by default, in a format customizable via the `table_format` argument.

In [ ]:
model_pred = cabinetry.model_utils.prediction(model)
_ = cabinetry.tabulate.yields(model_pred, data)

**Hint for your future** (aside)\
You can change the optimizer used for minimization using\
`pyhf.set_backend("numpy", pyhf.optimize.scipy_optimizer(*args, **kwargs))`\
or\
`pyhf.set_backend("numpy", pyhf.optimize.minuit_optimizer(*args, **kwargs))`.\
The `iminuit` minimizer used here can become unstable in some cases. If this happens to you, a first diagnostic test is switching to the `scipy` optimizer. This one is more stable, but will not return uncertainties on your fit parameters. If you find that `scipy` works, but `iminuit` fails, you can either try to get `iminuit` working by tuning settings such as the strategy or tolerance or fit using `scipy` first and then use the resulting best-fit parameters as input parameters to a second fit with `iminuit`.  

# More advanced features

Here we provide a list of more advanced features, for you to study if you have time. Most features are documented in one of these referenced:

* [`Cabinetry` Documentation](https://cabinetry.readthedocs.io/en/latest/index.html)
* [`Cabinetry` tutorial](https://github.com/cabinetry/cabinetry-tutorials/blob/master/example.ipynb)
* [Tutorial from Belle II `pyhf` workshop](https://github.com/alexander-held/Belle-II-cabinetry/blob/main/talk.ipynb)

## Asimov fit

If you do not have data points readily available, a good first check if your fit works properly is a fit to Asimov data. Find out what Asimov data is and how to obtain it (cabinetry can do this). Then perform a fit to the Asimov data and see if you obtain what you expect.

## Likelihood scan

Perform a likelihood scan of our POI. This will tell you about the (asymmetric) uncertainty of the parameter. Does it agree with the uncertainty obtained in the fit above? It will also tell you if your likelihood scan agrees well with a Gaussian approximation. Does it in this case? Can you explain why?

## Parameter ranking

We can rank nuisance parameters by their impact on the POI: how much does the POI change if the NP varies within its uncertainty? How is this determined? (This requires a lot of MLE fits.)
